# Models exploration 
- This notebook contains BERT model and MeanTeacher BERT model

Issue: 



TODO:

- Need to decide maxlen 
- ensemble technique
- better loss function
- ramp down function



# Requirement Installation 

In [32]:
#installing requirements
!pip install textattack==0.3.0

     |████████████████████████████████| 359 kB 9.2 MB/s 
     |████████████████████████████████| 20.1 MB 142 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: textattack
    Found existing installation: textattack 0.3.2
    Uninstalling textattack-0.3.2:
      Successfully uninstalled textattack-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.18.5 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 

# Importing and Declaration

In [2]:
# importing libraries 
import os 
import pandas as pd 
import numpy as np

import tensorflow as tf 
from tensorflow.keras.layers import LSTM, Bidirectional,Dropout, Dense, Embedding, Input 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,precision_recall_curve,precision_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt 

import transformers
from transformers import AutoTokenizer, TFAutoModel, TFDistilBertModel, TFBertModel

from pathlib import Path
import datetime

import random 
random.seed(42)

In [3]:
import os 
os.chdir('/content/drive/MyDrive/Master Thesis/Mean-Teacher-BERT-assessment/')

In [14]:

# Parameter declaration 
max_len= 100
learning_rate= 2e-5 
epochs= 3
batch_size=4
alpha=0.999
ratio=0.5

# columns names 
feature_col='tweet'
target_col='label'
aug_col='aug_tweet'
target_classes=['real','fake']
mapping_dict={target_classes[0]:0,target_classes[1]:1}

# Similarity Column 
cosine_sim='CosineSim'
wmd_sim='Wmd'
semantic_sim='Semantic_sim'


#pretrained_weights 
# pretrained_weights= 'bert-base-uncased'
pretrained_weights= 'distilbert-base-uncased'


#Model to test 1) DistllBERT_model.h5 2) BERT_model.h5
model_to_test= 'SavedModels/DistillBERT_model.h5'
model_to_test_teacher= 'SavedModels/DistillBERT_teacher.h5'

In [5]:
Data_loc= 'Data/codalab/'
# Training data location
df_train_loc=Data_loc+'PreprocessedData/pr_train.csv'
df_test_loc= Data_loc+'PreprocessedData/pr_test.csv' 
df_val_loc = Data_loc+'PreprocessedData/pr_val.csv'

# Adversarial unlabeled data location 
df_aug_syn_un_loc=Data_loc+'AugmentedData/aug_synonym.csv'
df_aug_con_un_loc=Data_loc+'AugmentedData/aug_context.csv'
df_aug_bt_un_loc=Data_loc+'AugmentedData/aug_synonym.csv'

# Adversarial test data 
df_aug_syn_test_loc=Data_loc+'AugmentedData/adv_test_syn.csv'
df_aug_con_test_loc=Data_loc+'AugmentedData/adv_test_context.csv'
df_aug_bt_test_loc=Data_loc+'AugmentedData/adv_test_backTrans.csv'


# Reading dataset 

In [6]:
def label_to_int(row):
    if row=='real':
        return 0
    else:
        return 1

In [7]:
# # training data 
# df_train= pd.read_csv(df_train_loc,index_col=[0])
# df_train= df_train.dropna()
# df_train= df_train.drop_duplicates().reset_index(drop=True)
# df_train[target_col]=df_train[target_col].map(lambda row: label_to_int(row))

# Test data 
df_test= pd.read_csv(df_test_loc,index_col=[0])
df_test= df_test.dropna()
df_test= df_test.drop_duplicates().reset_index(drop=True)
df_test[target_col]=df_test[target_col].map(lambda row: label_to_int(row))

# OPTIONAL: Validation data 
# df_val= pd.read_csv(df_val_loc,index_col=[0])
# df_val= df_val.dropna()
# df_val= df_val.drop_duplicates().reset_index(drop=True)
# df_val[target_col]=df_val[target_col].map(lambda row: label_to_int(row))


# df_train.shape, df_test.shape, df_val.shape

In [14]:
# Reading adversarial Unlabel data 
df_aug_syn_un= pd.read_csv(df_aug_syn_un_loc)
df_aug_syn_un= df_aug_syn_un.dropna().drop_duplicates().reset_index(drop=True)


df_aug_con_un= pd.read_csv(df_aug_con_un_loc)
df_aug_con_un= df_aug_con_un.dropna().drop_duplicates().reset_index(drop=True)

df_aug_bt_un= pd.read_csv(df_aug_bt_un_loc)
df_aug_bt_un= df_aug_bt_un.dropna().drop_duplicates().reset_index(drop=True)

# Combining all together 
df_aug_unlabel = df_aug_syn_un.append(df_aug_con_un).append(df_aug_bt_un)
df_aug_unlabel= df_aug_unlabel.sample(frac=1).reset_index(drop=True)
df_aug_unlabel[target_col]= df_aug_unlabel[aug_col].map(lambda row: label_to_int(row))

df_aug_syn_un.shape, df_aug_con_un.shape, df_aug_bt_un.shape, df_aug_unlabel.shape

((3946, 3), (1238, 3), (3946, 3), (9130, 3))

In [15]:
# Reading Adversarial Test data 
df_aug_syn_test= pd.read_csv(df_aug_syn_test_loc)
df_aug_syn_test= df_aug_syn_test.dropna().drop_duplicates().reset_index(drop=True)
df_aug_syn_test[target_col]=df_aug_syn_test[target_col].map(lambda row: label_to_int(row))

df_aug_con_test= pd.read_csv(df_aug_con_test_loc)
df_aug_con_test= df_aug_con_test.dropna().drop_duplicates().reset_index(drop=True)
df_aug_con_test[target_col]=df_aug_con_test[target_col].map(lambda row: label_to_int(row))

df_aug_bt_test= pd.read_csv(df_aug_bt_test_loc)
df_aug_bt_test= df_aug_bt_test.dropna().drop_duplicates().reset_index(drop=True)
df_aug_bt_test[target_col]=df_aug_bt_test[target_col].map(lambda row: label_to_int(row))

# Combining all together 
df_aug_test = df_aug_syn_test.append(df_aug_con_test).append(df_aug_bt_test)
df_aug_test= df_aug_test.sample(frac=1)

df_aug_syn_test.shape, df_aug_con_test.shape, df_aug_bt_test.shape, df_aug_test.shape

((1983, 6), (615, 6), (595, 6), (3193, 6))

# Functions Declaration


Functions :
- Data Splitter 
- Creating Tokenizer 
- Convert to category 
- Data Tokenization 
- Evaluation 
- Augmentation Evaluation

In [8]:
# in case we need to perform splitting 
def dataset_split(dataset,test_size):
    '''Split the dataframe into train and test '''
    df_train=dataset.sample(frac=(1-test_size),random_state=200) #random state is a seed value
    df_test=df.drop(df_train.index)
    df_train= df_train.reset_index(drop=True)
    df_test=df_test.reset_index(drop=True)
    return df_train,df_test

# Converting to categories 
def convert_to_category(target, n_classes=2):
    return to_categorical(target,n_classes)

#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-uncased'):
    '''Function to create the tokenizer'''

    tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
    return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_name,pretrained_weights,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_name].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)

    # if we have label column
    if (target_col in dataset.columns):
        y= convert_to_category(dataset[target_col],2)
        return [input_ids,attention_mask], y, tokenizer.vocab_size
    else:
        return [input_ids,attention_mask]

# Model Evaluation 
def evaluation(model,x_test,y_test,n_class=2):
    '''
    Evaluation of the model in case you have test data in numpy array format
    Return- predicted label and accuracy
    '''
    y_pred= model.predict(x_test)
    if n_class>1:
        y_pred=tf.argmax(y_pred,1)
        y_true=tf.argmax(y_test,1)
    else:
        y_true=y_test
    cl_r=classification_report(y_true,y_pred,output_dict=True)
    # print(classification_report(y_true,y_pred))
    return y_pred, cl_r['accuracy']


# Augmented Data Evaluation 
def augmentation_evaluation(df_aug,model,tokenizer, only_input_ids=False, output_result=False):
    '''
    Function to evaluate the augmented and orignial test data
    in case you have dataframe for prediction
    Only_input_ids: if you need only input ids from tokenizer
    output_result: True return the  original and augmentated accuracy
    along with the df
    Return: df with prediction and it will write in the report file
    '''
    try:
        if only_input_ids:
            [x_orig,_],y,_ = data_tokenization(df_aug,feature_col,pretrained_weights,max_len,tokenizer)
            [x_aug,_],_,_ = data_tokenization(df_aug,aug_col,pretrained_weights,max_len,tokenizer)
        else:
            x_orig,y,_ = data_tokenization(df_aug,feature_col,pretrained_weights,max_len,tokenizer)
            x_aug,_,_ = data_tokenization(df_aug,aug_col,pretrained_weights,max_len,tokenizer)
    except:
        pass
    df_result=df_aug.copy()
   
    y_pred,ori_acc=evaluation(model,x_orig,y)
    df_result['original_pred']= y_pred

    
    y_pred,aug_acc=evaluation(model,x_aug,y)
    df_result['aug_pred']=y_pred

    if output_result:
      return df_result,ori_acc,aug_acc
    else:
      return




# Report Writing

This section is related to writing required details in dataframe and saving in csv format.

In [ ]:
# creating report df dataframe
def create_report():
  '''
  Creating the dataframe for report
  '''
  report_df = pd.DataFrame(columns=['Date', 'Model','Train_size',
                                    'Aug_Test_size', 'Type','Hyperparameters',
                                    'Original_Accuracy','AA_Accuracy','Perturbation',
                                    'Avg_Sem_Sim','Avg_Cos_Similarity','Avg_WMD','Comment'])

  return report_df


#Logging the details
def report_writing(model,tokenizer,model_name,df_test,type_of_aug,comment,only_input=False):
  '''
  Function to write the evaluation metrics in the report csv file.
  Input:
  model- model to evaluate
  df_test- test dataframe, must have original text, aug text, label, cosine sim, wmd, similarity
  hyperparamter_details- epochs, batch_size, alpha, ratio
  TODO: Perturbation percentage
  '''
  # accessing the details
  df_result, ori_acc, aug_acc=augmentation_evaluation(df_test,model,tokenizer,only_input_ids=only_input, output_result=True)
  print(f'Original Accuracy : {ori_acc}, Accuracy after Attack : {aug_acc}')
  train_size=df_train.shape[0]
  test_size=df_test.shape[0]
  hyperparameters=[epochs,learning_rate,batch_size,alpha,ratio,max_len,pretrained_weights]
  avg_sem_sim=df_test[semantic_sim]


  #creating the report
  report_df=create_report()

  #adding in the dataframe
  x = datetime.datetime.now()
  report_df = report_df.append({'Date' : x.strftime("%c"), 'Model' :model_name,'Train_size': train_size ,
                                'Aug_Test_size': test_size, 'Type': type_of_aug,'Hyperparameters': hyperparameters,
                                'Original_Accuracy': ori_acc,'AA_Accuracy':aug_acc,'Perturbation': None,
                                'Avg_Sem_Sim': df_test[semantic_sim].mean(),'Avg_Cos_Similarity':df_test[cosine_sim].mean(),
                                'Avg_WMD':df_test[wmd_sim].mean(),'Comment':comment
                                  }, ignore_index=True)  
 
  file_l = Path('Result/'+model_name+'_'+'.csv')

  if file_l.exists():
      report_df.to_csv(file_l,mode='a', header= False , index = False)
  else:
      report_df.to_csv(file_l,mode='w', header= True , index= False)
  return 

In [ ]:
# Supervised model declaration 
def seq_model(maxlen,vocab_size):
    input= tf.keras.Input(shape=(max_len,))
    x= Embedding(vocab_size,max_len,input_length=None)(input)
    x= Bidirectional(LSTM(128))(x)
    x= Dense(64)(x)
    x= Dropout(0.2)(x)
    x=Dense(32)(x)
    x= Dense(16)(x)
    output=Dense(2,activation='sigmoid')(x)
    model= Model(input,output)
    model.compile(tf.keras.optimizers.Adam(learning_rate= learning_rate),loss= tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    return model

In [ ]:
# tokenization 
tokenizer= create_tokenizer()
[x_train,_],y_train,vocab_size= data_tokenization(df_train,feature_col,pretrained_weights,max_len,tokenizer)
[x_test,_],y_test,_= data_tokenization(df_test,feature_col,pretrained_weights,max_len,tokenizer)

In [ ]:
#training
bilstm_model=seq_model(max_len,vocab_size)
# bilstm_model.summary()
with tf.device('/GPU:0'):
    bilstm_model.fit(x_train,y_train,
                     batch_size=batch_size,
                     epochs=epochs+4,
                     verbose=1)

In [ ]:
report_writing(bilstm_model,tokenizer,'BILSTM',df_aug_syn_fil_test,'syn','Bilstm model', only_input=True)
report_writing(bilstm_model,tokenizer,'BILSTM',df_aug_con_fil_test,'con','Bilstm model', only_input= True)
report_writing(bilstm_model,tokenizer,'BILSTM',df_aug_bt_fil_test,'bt','Bilstm model', only_input= True)
report_writing(bilstm_model,tokenizer,'BILSTM',df_aug_fil_test,'all','Bilstm model', only_input=True)

In [ ]:
# saving the model
bilstm_model.save('SavedModels/Bilstm_model.h5')

In [ ]:
test_model=tf.keras.models.load_model('SavedModels/Bilstm_model.h5')

report_writing(test_model,tokenizer,'BILSTM',df_aug_syn_fil_test,'syn','Bilstm model', only_input=True)

# BERT Model 

In [9]:
from transformers import TFDistilBertModel, DistilBertConfig

def bert_model(max_len,pretrained_weights):
    '''BERT model creation with pretrained weights
    max_len: input length '''
    bert=TFAutoModel.from_pretrained(pretrained_weights)
    
    # This is must if you would like to train the layers of language models too.
    for layer in bert.layers:
        layer.trainable = True

    # parameter declaration
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # declaring inputs, BERT take input_ids and attention_mask as input
    input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
    attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

    bert= bert(input_ids,attention_mask=attention_mask)
    x= bert[0][:,0,:]
    x=tf.keras.layers.Dropout(0.2)(x)
    x= tf.keras.layers.Dense(64)(x)
    x=tf.keras.layers.Dense(32)(x)

    output=tf.keras.layers.Dense(2,activation='sigmoid')(x)

    model=Model(inputs=[input_ids,attention_mask],outputs=[output])
    # compiling model 
    model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
tokenizer=create_tokenizer(pretrained_weights)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col,pretrained_weights,max_len,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col,pretrained_weights,max_len,tokenizer)

In [ ]:
bert=bert_model(max_len,pretrained_weights)
bert.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           tf_bert_model[0][0]         

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/3
1523/1523 [==============================] - 189s 118ms/step - loss: 0.2555 - accuracy: 0.8961
Epoch 2/3
1523/1523 [==============================] - 183s 120ms/step - loss: 0.0869 - accuracy: 0.9668
Epoch 3/3
1520/1523 [============================>.] - ETA: 0s - loss: 0.0282 - accuracy: 0.9923

In [ ]:
evaluation(bert,x_test,y_test)

(<tf.Tensor: shape=(2040,), dtype=int64, numpy=array([0, 1, 1, ..., 0, 0, 0])>,
 0.9470588235294117)

In [ ]:
#saving the model 
bert.save_weights('SavedModels/BERT_model.h5')

# Adversarial Mean Teacher BERT

## Functions

In [ ]:
# ref:https://github.com/CuriousAI/mean-teacher/tree/master/tensorflow/mean_teacher  updated according to our need .
def cal_classification_cost(logits, labels) :
    """ Commputing classification cost , after removing labels -1 of unlabelled data and then calculating
    the binary cross entropy .
    """
    # applicable = tf.not_equal(labels, -1)
    # Change -1s to zeros to make cross-entropy computable
    # labels = tf.where(applicable, labels, tf.zeros_like(labels))
    loss=tf.reduce_sum(tf.keras.losses.categorical_crossentropy(labels, logits))
    # tf.keras.losses.kl_divergence()
    # Retain costs only for labeled
    # per_sample=tf.where(applicable[:,1], per_sample, tf.zeros_like(per_sample))
    # Take mean over all examples, not just labeled examples.
    # loss = tf.math.divide(tf.reduce_mean(tf.reduce_sum( per_sample ) ), np.shape ( per_sample )[0] )
    # print('Classification Cost: ',loss)
    return loss

def cal_overall_cost(x_train,y_train,x_unlabel,student, teacher):
    #TODO: need to include noising technique
    ratio=0.5
    logits = student(x_train)
    classification_cost = cal_classification_cost(logits, y_train)
    tar_student = student(x_unlabel)
    tar_teacher = teacher(x_unlabel)
    
    consistency_cost = cal_consistency_cost( tar_student, tar_teacher)
    # print('consistency_cost : ',consistency_cost)
    return (ratio * classification_cost) + ((1 - ratio) * consistency_cost)

# function for consistency cost
def cal_consistency_cost(student_output, teacher_output) :
    return tf.reduce_sum(tf.losses.mean_squared_error(student_output, teacher_output ))

def ema(student_model, teacher_model, alpha=0.999) :
    # taking weights
    student_weights = student_model.get_weights()
    teacher_weights = teacher_model.get_weights()
    # length must be equal otherwise it will not work
    assert len(student_weights ) == len(teacher_weights ), 'length of student and teachers weights are not equal Please check. \n Student: {}, \n Teacher:{}'.format (
        len(student_weights ), len (teacher_weights ) )
    new_layers = []
    for i, layers in enumerate ( student_weights ) :
        new_layer = alpha * (teacher_weights[i]) + (1 - alpha) * layers
        new_layers.append(new_layer)
    teacher_model.set_weights(new_layers)
    return teacher_model

def data_slices(x_train,y_train,x_unlabel,batch_size):
    train_dataset = tf.data.Dataset.from_tensor_slices( (x_train[0],x_train[1], y_train) )
    train_dataset = train_dataset.shuffle( buffer_size=1024 ).batch(batch_size)

    unlabel_dataset = tf.data.Dataset.from_tensor_slices( (x_unlabel[0],x_unlabel[1]) )
    unlabel_dataset = unlabel_dataset.shuffle( buffer_size=1024 ).batch(batch_size)
    return train_dataset, unlabel_dataset

In [ ]:
def train_mean_teacher(x_train, y_train, x_unlabel, epochs,batch_size,lr,alpha):
    # preparing the training dataset
    train_dataset,unlabel_dataset = data_slices(x_train, y_train,x_unlabel,batch_size)
    # declaring optimiser
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    # creating model
    student = bert_model(max_len,pretrained_weights)#call for the model here
    teacher = bert_model(max_len,pretrained_weights) #call for model here
    print('alpha:', alpha)

    # declaring metrics
    train_metrics = tf.keras.metrics.BinaryAccuracy(name='Binary_Accuracy')
    progbar = tf.keras.utils.Progbar(len(train_dataset), stateful_metrics=['Accuracy', 'Overall_Loss'])
    # epochs = args.epochs
    step_counter = 0
    for epoch in range(epochs):
        tf.print(f'\nepoch {epoch + 1}')
        # iterator_noise = iter(noise_dataset)
        for step, ((input_ids,attention_ids, y_batch_train),(input_ids_un,attention_ids_un)) in enumerate(zip(train_dataset,unlabel_dataset)):
            with tf.GradientTape() as tape:
                overall_cost = cal_overall_cost([input_ids,attention_ids], 
                                                y_batch_train,
                                                [input_ids_un,attention_ids_un],
                                                student, 
                                                teacher)

            grads = tape.gradient(overall_cost, student.trainable_weights)
            optimizer.apply_gradients((grad, var) for (grad, var) in zip(grads, student.trainable_weights) if grad is not None)
            # applying student weights to teacher
            step_counter += 1
            teacher = ema(student, teacher, alpha=alpha)
            # calculating training accuracy
            logits_t = teacher([input_ids,attention_ids])
            train_acc = train_metrics(tf.argmax(y_batch_train, 1), tf.argmax(logits_t, 1))
            progbar.update(step, values=[('Accuracy', train_acc), ('Overall_Loss', overall_cost)])

    return student, teacher


In [ ]:
# Training
tokenizer=create_tokenizer(pretrained_weights)
x_train,y_train,vocab_size= data_tokenization(df_train,feature_col,pretrained_weights,max_len,tokenizer)
x_test,y_test,_= data_tokenization(df_test,feature_col,pretrained_weights,max_len,tokenizer)
x_unlabel,_,_=data_tokenization(df_aug_unlabel,aug_col,pretrained_weights,max_len,tokenizer)

In [ ]:
with tf.device('/GPU:0'):
    student, teacher= train_mean_teacher(x_train, y_train, x_unlabel, 
                                         epochs,batch_size,
                                         learning_rate,alpha)

alpha: 0.999

epoch 1
1522/1523 [============================>.] - ETA: 2s - Accuracy: 0.8189 - Overall_Loss: 0.0305
epoch 2
1522/1523 [============================>.] - ETA: 2s - Accuracy: 0.9031 - Overall_Loss: 0.0066
epoch 3
1522/1523 [============================>.] - ETA: 2s - Accuracy: 0.9352 - Overall_Loss: 0.0026

In [ ]:
teacher.save_weights('SavedModels/BERT_teacher.h5')

In [ ]:
evaluation(teacher,x_test,y_test)

(<tf.Tensor: shape=(2040,), dtype=int64, numpy=array([0, 1, 1, ..., 0, 0, 0])>,
 0.9524509803921568)

In [ ]:
del bert

# Text attack

In [10]:
import numpy as np
import torch
import textattack
# model wrapper for attack
from textattack.models.wrappers import ModelWrapper
#attack recipe importing
from textattack.attack_recipes.bert_attack_li_2020 import BERTAttackLi2020
from textattack.attack_recipes import PWWSRen2019
from textattack.attack_recipes.bae_garg_2019 import BAEGarg2019
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019
from textattack import Attacker


class CustomTensorFlowModelWrapper(ModelWrapper):
    '''
    This wrapper is required to attack the model using text attack.
    '''
    def __init__(self, model,pretrained_weights):
        self.model = model
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)

    def __call__(self, text_input_list):
        tokens=self.tokenizer(text_input_list,return_tensors='tf', 
                         truncation=True,
                         padding='max_length',
                         max_length=max_len, 
                         add_special_tokens=True)
        
        input_ids= []
        attention_mask=[]
        input_ids.append(tokens.input_ids)
        attention_mask.append(tokens.attention_mask)
        preds = torch.tensor(self.model([input_ids, attention_mask]).numpy())
  
        return preds


In [11]:
df_test['len']=df_test.apply(lambda row: len(row[feature_col].split()),axis=1)
df_test_for_attack= df_test[(df_test['len']>0) & (df_test['len']<100)][[feature_col,target_col]].dropna().reset_index(drop=True)
df_test_for_attack=df_test_for_attack.sample(frac=1).reset_index(drop=True)

df_test_for_attack

,tweet,label
0,it didn t have to be this bad donald trump utt...,1
1,rt pmoindia the bed hospital at bihta patna wi...,0
2,a of june more than million covid case have be...,0
3,a tweet from donald trump said that he would n...,1
4,coronavirusupdates indiafightscorona covid rec...,0
...,...,...
2035,doctor operated and took out the kidney of a c...,1
2036,maximum suppression is our strategy fmwales sa...,0
2037,the latest cdc covidview data show that covida...,0
2038,our update is published major caveat to the da...,0


In [12]:
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H:%M")

#folder name for saving the report 
folderName= 'Textfooler'#'BAE'#'BERTAttackBert' #PWWSRen2019

#creating dataset for the attacking
dataset_for_attack=list(df_test_for_attack.itertuples(index=False, name=None))
dataset_for_attack = textattack.datasets.Dataset(dataset_for_attack)

In [15]:
#loading the model
BERT_model= bert_model(max_len,pretrained_weights) 
BERT_model.load_weights(model_to_test)

model_wrapper = CustomTensorFlowModelWrapper(BERT_model,pretrained_weights)

# attack = PWWSRen2019.build(model_wrapper)
# attack = BERTAttackLi2020.build(model_wrapper)
# attack=BAEGarg2019.build(model_wrapper)
attack=TextFoolerJin2019.build(model_wrapper)
attack_args = textattack.AttackArgs (
    num_examples=500,
    log_to_csv=f'Result/{folderName}/DistillBert_nexp500_len100_qb200_{now}.csv',
    disable_stdout=False,
    query_budget=200
)
with tf.device('/GPU:0'):
  attacker = Attacker(attack, dataset_for_attack,attack_args)
  attacker.attack_dataset()


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path Result/Textfooler/DistillBert_nexp500_len100_qb200_2021-07-29_20:26.csv
  0%|          | 0/500 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

...

--------------------------------------------- Result 498 ---------------------------------------------
0 (73%) --> [FAILED]

indiafightscorona a india record more recovery than the new case the gap between recovered case and the active case is continuously widening the recovery are more than time active case




[Succeeded / Failed / Skipped / Total] 342 / 133 / 24 / 499: 100%|█████████▉| 499/500 [07:34<00:00,  1.10it/s]

--------------------------------------------- Result 499 ---------------------------------------------
0 (69%) --> 1 (68%)

abusedeterrent oxycontin will not solve the opioid epidemic

abusedeterrent oxycontin will not fixes the opioid smallpox




[Succeeded / Failed / Skipped / Total] 343 / 133 / 24 / 500: 100%|██████████| 500/500 [07:35<00:00,  1.10it/s]

--------------------------------------------- Result 500 ---------------------------------------------
0 (73%) --> 1 (51%)

a study from the annals of internal medicine estimate that the probability of pcr test resulting in falsenegative varies over the course of the illness the highest risk being before symptom appear and lowest three day after symptom appear

a study from the annals of internal medicine calculating that the probability of pcr test resulting in falsenegative differ over the course of the illness the highest risk being before symptom spectacle and lowest three day after symptom spectacle



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 343    |
| Number of failed attacks:     | 133    |
| Number of skipped attacks:    | 24     |
| Original accuracy:            | 95.2%  |
| Accuracy under attack:        | 26.6%  |
| Attack success rate:          | 72.06% 

In [16]:
from textattack.loggers import CSVLogger
from textattack.shared import AttackedText, logger
l=CSVLogger()
l.flush()

textattack: Logging to CSV at path results.csv


In [17]:
import gc
del BERT_model
tf.keras.backend.clear_session()
gc.collect()

393

In [18]:
BERT_teacher= bert_model(max_len,pretrained_weights) 

BERT_teacher.load_weights(model_to_test_teacher)

model_wrapper = CustomTensorFlowModelWrapper(BERT_teacher,pretrained_weights)


# attack = PWWSRen2019.build(model_wrapper)
# attack = BERTAttackLi2020.build(model_wrapper)
#attack=BAEGarg2019.build(model_wrapper)
attack=TextFoolerJin2019.build(model_wrapper)
attack_args = textattack.AttackArgs(
    num_examples=500,
    log_to_csv=f'Result/{folderName}/DBTeacher_nexp500_len100_qb200_{now}.csv',
    disable_stdout=False,
    query_budget=200
)
with tf.device('/GPU:0'):
  attacker = Attacker(attack, dataset_for_attack,attack_args)
  attacker.attack_dataset()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path Result/Textfooler/DBTeacher_nexp500_len100_qb200_2021-07-29_20:26.csv
  0%|          | 0/500 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/500 [00:04<36:02,  4.33s/it]

...

--------------------------------------------- Result 497 ---------------------------------------------
1 (50%) --> 0 (52%)

anyone in mumbai requiring plasma for covid treatment please contact from a group of people ready to make plasma donation

anyone in pune requiring plasma for covid treatment please contact from a group of people ready to make plasma donation




[Succeeded / Failed / Skipped / Total] 215 / 264 / 20 / 499: 100%|█████████▉| 499/500 [08:17<00:00,  1.00it/s]

--------------------------------------------- Result 498 ---------------------------------------------
0 (68%) --> [FAILED]

indiafightscorona a india record more recovery than the new case the gap between recovered case and the active case is continuously widening the recovery are more than time active case


--------------------------------------------- Result 499 ---------------------------------------------
1 (57%) --> [SKIPPED]

abusedeterrent oxycontin will not solve the opioid epidemic




[Succeeded / Failed / Skipped / Total] 216 / 264 / 20 / 500: 100%|██████████| 500/500 [08:18<00:00,  1.00it/s]

--------------------------------------------- Result 500 ---------------------------------------------
0 (64%) --> 1 (52%)

a study from the annals of internal medicine estimate that the probability of pcr test resulting in falsenegative varies over the course of the illness the highest risk being before symptom appear and lowest three day after symptom appear

a study from the annals of internal medicine estimate that the probability of pcr test resulting in falsenegative varies over the course of the illness the highest risque being before symptom appear and lowest three day after symptom visualized



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 216    |
| Number of failed attacks:     | 264    |
| Number of skipped attacks:    | 20     |
| Original accuracy:            | 96.0%  |
| Accuracy under attack:        | 52.8%  |
| Attack success rate:          | 45.0%  |
|

In [ ]:
import gc
del BERT_teacher
tf.keras.backend.clear_session()
gc.collect()

15204